# Stock Trades by Members of the US House of Representatives

This project uses public data about the stock trades made by members of the US House of Representatives. This data is collected and maintained by Timothy Carambat as part of the [House Stock Watcher](https://housestockwatcher.com/) project. The project describes itself as follows:

> With recent and ongoing investigations of incumbent congressional members being investigated for potentially violating the STOCK act. This website compiles this publicly available information in a format that is easier to digest then the original PDF source.
>
> Members of Congress must report periodic reports of their asset transactions. This website is purely for an informative purpose and aid in transparency.
>
> This site does not manipluate or censor any of the information from the original source. All data is transcribed by our community of contributors, which you can join for free by going to our transcription tool. Our moderation team takes great care in ensuring the accuracy of the information.
>
> This site is built and maintained by Timothy Carambat and supported with our contributors.

In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
import requests
import bs4
\
import lxml

Some interesting questions to consider for this data set include:

- Is there a difference in stock trading behavior between political parties? For example:
    - does one party trade more often?
    - does one party make larger trades?
    - do the two parties invest in different stocks or sectors? For instance, do Democrats invest in Tesla more than Republicans?
- What congresspeople have made the most trades?
- What companies are most traded by congresspeople?
- Is there evidence of insider trading? For example, Boeing stock dropped sharply in February 2020. Were there a suspiciously-high number of sales of Boeing before the drop?
- When are stocks bought and sold? Is there a day of the week that is most common? Or a month of the year?

### Getting the Data

The full data set of stock trade disclosures is available as a CSV or as JSON at https://housestockwatcher.com/api.

This data set does not, however, contain the political affiliation of the congresspeople. If you wish to investigate a question that relies on having this information, you'll need to find another dataset that contains it and perform a merge. *Hint*: Kaggle is a useful source of data sets.

In [3]:
r = requests.get('https://house-stock-watcher-data.s3-us-west-2.amazonaws.com/data/all_transactions.json')
df = pd.DataFrame(r.json())

In [4]:
sort_date = df.sort_values(by='disclosure_year', ascending = False)
sort_date.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15699 entries, 10706 to 15698
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   disclosure_year         15699 non-null  int64 
 1   disclosure_date         15699 non-null  object
 2   transaction_date        15699 non-null  object
 3   owner                   9667 non-null   object
 4   ticker                  15699 non-null  object
 5   asset_description       15695 non-null  object
 6   type                    15699 non-null  object
 7   amount                  15699 non-null  object
 8   representative          15699 non-null  object
 9   district                15699 non-null  object
 10  ptr_link                15699 non-null  object
 11  cap_gains_over_200_usd  15699 non-null  bool  
dtypes: bool(1), int64(1), object(10)
memory usage: 1.5+ MB


In [5]:
sort_date

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd
10706,2022,10/11/2022,2022-09-12,self,TSLA,Tesla Inc,sale_partial,"$1,001 - $15,000",Hon. Kathy Manning,NC06,https://disclosures-clerk.house.gov/public_dis...,False
7628,2022,09/16/2022,2022-08-11,joint,CCXI,ChemoCentryx Inc,sale_partial,"$1,001 - $15,000",Hon. Josh Gottheimer,NJ05,https://disclosures-clerk.house.gov/public_dis...,False
7570,2022,09/02/2022,2021-07-28,None,MRK,Merck & Company Inc,sale_partial,"$15,001 - $50,000",Hon. Carol Devine Miller,WV03,https://disclosures-clerk.house.gov/public_dis...,False
7569,2022,09/02/2022,2021-11-10,self,LHX,L3Harrie Technologies Inc,sale_partial,"$15,001 - $50,000",Hon. Carol Devine Miller,WV03,https://disclosures-clerk.house.gov/public_dis...,False
7568,2022,09/02/2022,2021-03-04,self,JNJ,Johnson & Johnson,purchase,"$15,001 - $50,000",Hon. Carol Devine Miller,WV03,https://disclosures-clerk.house.gov/public_dis...,False
...,...,...,...,...,...,...,...,...,...,...,...,...
7377,2020,08/07/2020,2020-05-28,self,CRM,Salesforce.com Inc,purchase,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,False
7378,2020,08/07/2020,2020-06-29,self,SFST,"Southern First Bancshares, Inc.",sale_full,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,False
7379,2020,08/07/2020,2020-05-28,self,SBUX,Starbucks Corporation,sale_full,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,True
7380,2020,08/07/2020,2020-05-28,self,WMT,Walmart Inc.,purchase,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,False


In [12]:
affiliation = pd.read_csv("house_members_116.csv")
affiliate = affiliation[['name', 'current_party']]
affiliate

,name,current_party
0,ralph-abraham,Republican
1,alma-adams,Democratic
2,robert-aderholt,Republican
3,pete-aguilar,Democratic
4,rick-allen,Republican
...,...,...
438,ron-wright,Republican
439,john-yarmuth,Democratic
440,ted-yoho,Republican
441,don-young,Republican


In [17]:
affiliate['name'] = affiliate['name'].str.replace('-', ' ')
affiliate

C:\Users\ChristineXu\AppData\Local\Temp\ipykernel_2212\1128786723.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affiliate['name'] = affiliate['name'].str.replace('-', ' ')


,name,current_party
0,ralph abraham,Republican
1,alma adams,Democratic
2,robert aderholt,Republican
3,pete aguilar,Democratic
4,rick allen,Republican
...,...,...
438,ron wright,Republican
439,john yarmuth,Democratic
440,ted yoho,Republican
441,don young,Republican


In [67]:
def clean_name(name):
    name = name.strip('Hon. ')
    no_middle = name.split(' ')[0] +' ' + name.split(' ')[-1]
    return no_middle.lower()
affiliate['cleaned_name'] = affiliate['name'].transform(clean_name)
affiliate

C:\Users\ChristineXu\AppData\Local\Temp\ipykernel_2212\3347264736.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affiliate['cleaned_name'] = affiliate['name'].transform(clean_name)


,name,current_party,cleaned_name
0,ralph abraham,Republican,ralph abraham
1,alma adams,Democratic,alma adams
2,robert aderholt,Republican,robert aderholt
3,pete aguilar,Democratic,pete aguilar
4,rick allen,Republican,rick alle
...,...,...,...
438,ron wright,Republican,ron wright
439,john yarmuth,Democratic,john yarmuth
440,ted yoho,Republican,ted yoh
441,don young,Republican,don young


In [29]:
name = 'Carol Devine Miller'
no_middle = name.split(' ')[0] +' ' + name.split(' ')[-1]
no_middle.lower()

'carol miller'

In [71]:
sort_date['cleaned_name'] = sort_date['representative'].transform(clean_name)
with_party = pd.merge(sort_date, affiliate, on ='cleaned_name', how='left')
'kathy manning' in with_party[with_party['name'].isna()]['cleaned_name'].values
with_party

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd,cleaned_name,name,current_party
0,2022,10/11/2022,2022-09-12,self,TSLA,Tesla Inc,sale_partial,"$1,001 - $15,000",Hon. Kathy Manning,NC06,https://disclosures-clerk.house.gov/public_dis...,False,kathy manning,NaN,NaN
1,2022,09/16/2022,2022-08-11,joint,CCXI,ChemoCentryx Inc,sale_partial,"$1,001 - $15,000",Hon. Josh Gottheimer,NJ05,https://disclosures-clerk.house.gov/public_dis...,False,josh gottheimer,josh gottheimer,Democratic
2,2022,09/02/2022,2021-07-28,None,MRK,Merck & Company Inc,sale_partial,"$15,001 - $50,000",Hon. Carol Devine Miller,WV03,https://disclosures-clerk.house.gov/public_dis...,False,carol miller,carol miller,Republican
3,2022,09/02/2022,2021-11-10,self,LHX,L3Harrie Technologies Inc,sale_partial,"$15,001 - $50,000",Hon. Carol Devine Miller,WV03,https://disclosures-clerk.house.gov/public_dis...,False,carol miller,carol miller,Republican
4,2022,09/02/2022,2021-03-04,self,JNJ,Johnson & Johnson,purchase,"$15,001 - $50,000",Hon. Carol Devine Miller,WV03,https://disclosures-clerk.house.gov/public_dis...,False,carol miller,carol miller,Republican
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15694,2020,08/07/2020,2020-05-28,self,CRM,Salesforce.com Inc,purchase,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,False,william timmons,william timmons,Republican
15695,2020,08/07/2020,2020-06-29,self,SFST,"Southern First Bancshares, Inc.",sale_full,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,False,william timmons,william timmons,Republican
15696,2020,08/07/2020,2020-05-28,self,SBUX,Starbucks Corporation,sale_full,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,True,william timmons,william timmons,Republican
15697,2020,08/07/2020,2020-05-28,self,WMT,Walmart Inc.,purchase,"$1,001 - $15,000",Hon. William R. Timmons,SC04,https://disclosures-clerk.house.gov/public_dis...,False,william timmons,william timmons,Republican


In [74]:
nans = with_party[with_party['name'].isna()]['cleaned_name'].unique()
nans

array(['kathy manning', 'cindy axne', 'richard alle', 'scott frankli',
       'david cawthor', 'maria salazar', 'christopher jacobs',
       'nancy pelosi', 'marjorie greene', 'bill pascrell', 'greg steube',
       'august pfluger', 'pete sessions', 'daniel crenshaw',
       'andrew garbari', 'victoria spartz', 'patrick fall',
       'nicholas taylor', 'greg murphy', 'jake auchincloss',
       'arold rogers', 'deborah ross', 'marie newma', 'blake moore',
       'michael garcia', 'diana harshbarger', 'sara jacobs',
       'peter meijer', 'neal du', 'james costa', 'rohit khanna',
       'michael gallagher', 'none jacobs', 'k. conaway', 'aston mceachi',
       'arley rouda', 'kenneth buck', 'james banks', 'none arenholz',
       'none newma', 'none ross', 'felix moore', 'james hill',
       'mrs. greene', 'mr. meijer', 'stephanie bice', 'none manning',
       'akeem jeffries', 'raúl grijalva', 'none spartz', 'mr. frankli',
       'mario diaz-balart', 'w. steube', 'neal facs', 'james haged

In [62]:
'william timmons' in affiliate['name'].values

True

In [ ]:
replace = {'doug lamborn': 'doug lambor',
          'jim costa': 'james costa',
          'robert wittman': 'robert wittma',
          'steve cohen': 'steve cohe',
          'charles fleischmann': 'charles fleischma',
           ''
          }


### Cleaning and EDA

- Clean the data.
    - Certain fields have "missing" data that isn't labeled as missing. For example, there are fields with the value "--." Do some exploration to find those values and convert them to null values.
    - You may also want to clean up the date columns to enable time-series exploration.
- Understand the data in ways relevant to your question using univariate and bivariate analysis of the data as well as aggregations.


### Assessment of Missingness

- Assess the missingness per the requirements in `project03.ipynb`

### Hypothesis Test / Permutation Test
Find a hypothesis test or permutation test to perform. You can use the questions at the top of the notebook for inspiration.

# Summary of Findings

### Introduction
TODO

### Cleaning and EDA
TODO

### Assessment of Missingness
TODO

### Hypothesis Test
TODO

# Code

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

### Cleaning and EDA

In [ ]:
# TODO

### Assessment of Missingness

In [ ]:
# TODO

### Hypothesis Test / Permutation Test

In [ ]:
# TODO